In [21]:
import pandas as pd
import numpy as np
import os
import json
from calculate_ppda import calculate_match_ppda

In [30]:
DATA_DIR = os.path.join(".", 'data', 'wyscout_data')
league = 'Spain'
matches_dir = os.path.join(DATA_DIR, 'refined_events', league)

In [ ]:
teams_path = os.path.join(DATA_DIR, "teams.json")
with open(teams_path) as json_data:
    teams = json.load(json_data)
spain_teams = [team for team in teams if team.get('area', {}).get('name') == 'Spain']
pd.DataFrame(spain_teams)

In [31]:
match_files = [f for f in os.listdir(matches_dir) if f.endswith('.pkl')]
print(f"{league}: {len(match_files)}")

Spain: 380


In [32]:
# 모든 경기에 대해 PPDA를 계산하고 결과를 리스트에 저장
all_ppda_records = []
for match_file in match_files:
	match_file_path = os.path.join(matches_dir, match_file)
	match_events = pd.read_pickle(match_file_path)

	ppda_values = calculate_match_ppda(match_events)

	for team_id, ppda in ppda_values.items():
		team_name = match_events[match_events['team_id'] == team_id]['team_name'].iloc[0]
		all_ppda_records.append({
			'team_id': team_id,
			'team_name': team_name,
			'ppda': ppda
		})

ppda_df = pd.DataFrame(all_ppda_records)

In [33]:
# 팀별로 그룹화하여 평균 PPDA 계산
if not ppda_df.empty:
	team_avg_ppda = ppda_df.dropna(subset=['ppda']).groupby('team_name')['ppda'].mean().sort_values(ascending=True)

	print("\n--- 스페인 리그 팀별 평균 PPDA (압박 강도가 강한 순) ---")
	print(team_avg_ppda)
else:
	print("\nPPDA를 계산할 데이터가 없습니다.")


--- 스페인 리그 팀별 평균 PPDA (압박 강도가 강한 순) ---
team_name
Girona                 30.790539
Getafe                 31.284451
Villarreal             34.174020
Eibar                  34.975428
Sevilla                36.157534
Málaga                 36.789577
Las Palmas             36.878120
Deportivo Alavés       37.316347
Leganés                38.659702
Real Sociedad          38.717978
Real Betis             39.210748
Atlético Madrid        39.557620
Athletic Club          39.728718
Real Madrid            40.666100
Celta de Vigo          41.276239
Deportivo La Coruña    41.782835
Levante                42.055091
Espanyol               44.108385
Barcelona              47.072504
Valencia               47.258385
Name: ppda, dtype: float64
